| Dash Example Index | https://dash-example-index.herokuapp.com/ |
|--------------------|-------------------------------------------|



In [ ]:
import dash
from dash import Dash, dcc, html, Input, Output, dash_table, callback
import dash_mantine_components as dmc
import plotly.express as px
from dash import Dash, dcc, Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
import pandas as pd

In [ ]:
data = px.data.stocks()
data

In [ ]:
app = Dash(__name__)

app.layout = dmc.Container(
    [
        dmc.Title(
            "Equity prices - Line chart and Table data", align="center"),
        dmc.Space(h=20),
        dmc.Button("Download Table Data", id="btn_csv"),
        dcc.Download(id="download-dataframe-csv"),
        dmc.Space(h=10),
        dmc.MultiSelect(
            label="Select stock you like!",
            placeholder="Select all stocks you like!",
            id="stock-dropdown",
            value=["GOOG", "AAPL"],
            data=[{"label": i, "value": i} for i in data.columns[1:]],
        ),
        dmc.Space(h=60),
        dmc.SimpleGrid(
            [
                dcc.Graph(id="line_chart"),
                dash_table.DataTable(
                    data.to_dict("records"),
                    [{"name": i, "id": i} for i in data.columns],
                    page_size=10,
                    style_table={"overflow-x": "auto"},
                ),
            ],
            cols=2,
            id="simple_grid_layout",
            breakpoints=[
                {"maxWidth": 1500, "cols": 2, "spacing": "md"},
                {
                    "maxWidth": 992,
                    "cols": 1,
                    "spacing": "sm",
                },  # common screen size for small laptops
                {
                    "maxWidth": 768,
                    "cols": 1,
                    "spacing": "sm",
                },  # common screen size for tablets
            ],
        ),
    ],
    fluid=True,
)


@callback(
    Output("line_chart", "figure"),
    Input("stock-dropdown", "value"),
)
def select_stocks(stocks):
    fig = px.line(data_frame=data, x="date", y=stocks, template="simple_white")
    fig.update_layout(
        margin=dict(t=50, l=25, r=25, b=25), yaxis_title="Price", xaxis_title="Date"
    )
    return fig


@callback(
    Output("download-dataframe-csv", "data"),
    Input("btn_csv", "n_clicks"),
    prevent_initial_call=True,
)
def func(n_clicks):
    return dcc.send_data_frame(data.to_csv, "mydf.csv")


if __name__ == "__main__":
    app.run_server(debug=True)

In [ ]:
df = pd.read_table(
    "https://raw.githubusercontent.com/plotly/datasets/master/global_super_store_orders.tsv"
)
df = df[df["Country"].isin(["China", "Germany", "United States", "France"])]
df["ship_date"] = pd.to_datetime(df["Ship Date"])

# convert shipping costs and profit from strings to floats
df["Shipping Cost"] = df["Shipping Cost"].str.replace(",", ".").astype("float")


app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container(
    [
        dcc.Markdown(
            "#### Shipping costs breakdown by country and category",
            style={"textAlign": "center"},
            className="my-4",
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        dcc.DatePickerRange(
                            id="datepicker",
                            min_date_allowed=min(df["ship_date"]),
                            max_date_allowed=max(df["ship_date"]),
                            end_date=max(df["ship_date"]),
                            start_date=min(df["ship_date"]),
                            clearable=False,
                        ),
                    ],
                    width=4,
                ),
                dbc.Col(
                    [
                        dcc.Tabs(
                            id="tabs",
                            value="tab-1",
                            children=[
                                dcc.Tab(
                                    label="Country",
                                    value="tab-1",
                                    children=[dcc.Graph(id="pie1")],
                                ),
                                dcc.Tab(
                                    label="Category",
                                    value="tab-2",
                                    children=[dcc.Graph(id="pie2")],
                                ),
                            ],
                        ),
                    ],
                    width=8,
                ),
            ]
        ),
    ]
)


@app.callback(
    Output("pie1", "figure"),
    Output("pie2", "figure"),
    Input("datepicker", "start_date"),
    Input("datepicker", "end_date"),
)
def render_content(start_date, end_date):
    dff = df.query("ship_date > @start_date & ship_date < @end_date")

    pie1_fig = px.pie(dff, values="Shipping Cost", names="Country")

    pie2_fig = px.pie(dff, values="Shipping Cost", names="Category")

    return (pie1_fig, pie2_fig)


if __name__ == "__main__":
    app.run_server(debug=True)

In [ ]:
df = px.data.tips()

app = Dash(__name__)

app.layout = html.Div(
    [
        html.H4("Analysis of the restaurant sales"),
        dcc.Graph(id="graph"),
        html.P("Names:"),
        dcc.Dropdown(
            id="names",
            options=["smoker", "day", "time", "sex"],
            value="day",
            clearable=False,
        ),
        html.P("Values:"),
        dcc.Dropdown(
            id="values",
            options=["total_bill", "tip", "size"],
            value="total_bill",
            clearable=False,
        ),
    ]
)

@app.callback(
    Output("graph", "figure"),
    Input("names", "value"),
    Input("values", "value"),
)
def generate_chart(names, values):
    fig = px.pie(df, values=values, names=names, hole=0.3)
    return fig


if __name__ == "__main__":
    app.run_server(debug=True)

In [ ]:
app = Dash(__name__)


app.layout = html.Div(
    [
        html.H4("Interactive data-scaling using the secondary axis"),
        html.P("Select red line's Y-axis:"),
        dcc.RadioItems(
            id="radio",
            options=["Primary", "Secondary"],
            value="Secondary",
        ),
        dcc.Graph(id="graph"),
    ]
)


@app.callback(
    Output("graph", "figure"), Input("radio", "value")
)
def display_(radio_value):
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(
            x=[1, 2, 3],
            y=[40, 50, 60],  # replace with your own data source
            name="yaxis data",
        ),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=[2, 3, 4], y=[4, 5, 6], name="yaxis2 data"),
        secondary_y=radio_value == "Secondary",
    )

    # Add figure title
    fig.update_layout(title_text="Double Y Axis Example")

    # Set x-axis title
    fig.update_xaxes(title_text="xaxis title")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
    fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

    return fig


if __name__ == "__main__":
    app.run_server(debug=True)

In [ ]:
# Reading the data and filling missing data
df = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/hofstede-cultural-dimensions.csv",
    delimiter=";",
)
df.replace(to_replace="#NULL!", value=0, inplace=True)

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


# Country selection card
controls = dbc.Card(
    [
        html.Div(
            [
                dbc.Label("Select Country"),
                dcc.Dropdown(
                    id="country",
                    options=[{"label": ctr, "value": ctr} for ctr in df["country"]],
                    value=["U.S.A.", "Turkey", "Korea South"],
                    multi=True,
                ),
            ]
        )
    ]
)

# Description card
tabs = dbc.Card(
    dbc.CardBody(
        dbc.Tabs(
            [
                dbc.Tab(
                    "The power distance index considers the extent to which inequality and power are tolerated. In this dimension, inequality and power are viewed from the viewpoint of the followers – the lower level.",
                    label="Power Distance",
                ),
                dbc.Tab(
                    "The individualism vs. collectivism dimension considers the degree to which societies are integrated into groups and their perceived obligations and dependence on groups.",
                    label="Individualism",
                ),
                dbc.Tab(
                    "The masculinity vs. femininity dimension is also referred to as “tough vs. tender,” and considers the preference of society for achievement, attitude towards sexuality equality, behavior, etc.",
                    label="Masculinity",
                ),
                dbc.Tab(
                    "The uncertainty avoidance index considers the extent to which uncertainty and ambiguity are tolerated. This dimension considers how unknown situations and unexpected events are dealt with.",
                    label="Uncertainty Avoidance",
                ),
                dbc.Tab(
                    "The long-term orientation vs. short-term orientation dimension considers the extent to which society views its time horizon.",
                    label="Long-Term Orientation",
                ),
                dbc.Tab(
                    "The indulgence vs. restraint dimension considers the extent and tendency for a society to fulfill its desires. In other words, this dimension revolves around how societies can control their impulses and desires.",
                    label="Indulgence",
                ),
            ]
        )
    )
)


app.layout = dbc.Container(
    [
        html.H1("Hofstede's Cultural Dimensions"),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col([controls, html.Br(), tabs], lg=4, sm=12),
                dbc.Col(dcc.Graph(id="graph-country"), lg=8, sm=12),
            ],
            align="center",
        ),
    ]
)


# Update Bar Chart Callback Function
@app.callback(Output("graph-country", "figure"), Input("country", "value"))
def make_country_graph(country_list):
    dff = df[df.country.isin(country_list)]

    fig = go.Figure()

    for country in dff.country:
        dft = dff[dff.country == country].reset_index()
        dft = dft.iloc[:, 3:].T.astype(int)
        fig.add_trace(
            go.Bar(
                x=[
                    "Power Distance",
                    "Individualism",
                    "Masculinity",
                    "Uncertainty Avoidance",
                    "Long Term Orientation",
                    "Indulgence",
                ],
                y=dft.iloc[:, 0],
                name=country,
            )
        )

    fig.update_layout(xaxis_tickfont_size=10)

    return fig


if __name__ == "__main__":
    app.run_server(debug=True)

In [2]:
from data.dhs_data import *

In [7]:
#dhs_daily_df
#monthly_avg_df
#quarterly_avg_df
#yearly_avg_df